In [ ]:
from ase.geometry import get_distances
import numpy as np
import ase.db
import pandas as pd
import networkx as nx
%matplotlib inline
import warnings
from adsorptionGraphGenerator import adsGraphGen

warnings.filterwarnings(action='ignore')

# Calculate reference energies

In [ ]:
# From DFT
E_H2Og = -.14220313E+02
E_H2g = -6.7709484

# Derived values
E_Hg = E_H2g/2.

# O2 gas correction
deltaG = -4.708
TdeltaS = -0.7024
deltaE_ZPE = 0.48
deltaH = deltaG+TdeltaS-deltaE_ZPE

E_O2g = 2*E_H2Og-2*E_H2g-deltaH
E_Og = E_O2g/2.
print('Corrected O2 gas in eV = ',E_O2g)
# adsorbate compositions
E_gas = {'H2': E_H2g, 'H2O':E_H2Og,'H':E_Hg,'O':E_Og}

# To get clean energies
fname = 'Oads_Mo2C_pristine_MDF.db'
db  = ase.db.connect(fname)

indexItermList = []
for row in db.select('fmax<=0.05'): # collect clean energies from converged systems
    index=row.data.index
    iterm=row.data.iterm
    indexIterm = index+'_'+iterm
    if indexIterm not in indexItermList:
        indexItermList.append(indexIterm)
        
E_clean = dict.fromkeys(indexItermList)
cleanSurfaceList = []

for row in db.select('fmax<=0.05'): # collect clean energies from converged systems
    if row.data.adsorbate=='clean':
        idx = row.data.index
        iterm = row.data.iterm
        idxIterm = idx+'_'+iterm
        e = row.energy
        E_clean[idxIterm]=e
        cleanSurfaceList.append((row.id,idx,iterm,e,'Mo2C_'+idxIterm))
print(E_clean)
print(len(E_clean))
dfCleanSurface = pd.DataFrame(cleanSurfaceList,columns=['databaseRowID','Index','Termination','TotalE','Name'])

# Compute binding energies and generate graphs

In [ ]:
import os
import time
start_time = time.time()

X = []
y_bindingE = []
y_totalE = []
rowID = []
surfaceIndex=[]
surfaceTerminations=[]
adsDistinctList = []
adsList = []
adsIsomerList = []
adsConfigurationList = []
adsIndicesList = []
distMaxList = []
distMaxAdsList = []
graphNameList = []

fname = 'Oads_Mo2C_pristine_MDF.db'
db  = ase.db.connect(fname)
features = []
iniDir = 'Oads_Mo2C_pristine_ini_graphml'
finDir = 'Oads_Mo2C_pristine_fin_graphml'
os.mkdir(iniDir)
os.mkdir(finDir)
for row in db.select('fmax<=0.05'):
    idx = row.data.index
    iterm = row.data.iterm
    idxIterm = idx+'_'+iterm
    BE= row.energy-E_clean[idxIterm]-E_gas[row.data.adsorbate]
    finAtoms = db.get_atoms(row.id)
    iniAtoms = db.get_atoms(row.id)
    iniPos = db.get(row.id).data.initialPositions
    adsIndices = db.get(row.id).data.adsorbateIndices
    iniAtoms.set_positions(iniPos)
    finAds = finAtoms[adsIndices[0]:]
    pwdistTup = get_distances(iniAtoms.positions,finAtoms.positions,cell=iniAtoms.get_cell(),pbc=True)
    pwdist = np.diagonal(pwdistTup[1])
    distMax = np.amax(pwdist)
    if len(adsIndices)>1:  # Measure distance(s) from O to H(s)
        pwdistAds = finAtoms.get_distances(a=adsIndices[0],indices=adsIndices[1:])
        distMaxAds = np.amax(pwdistAds)
    else:
        distMaxAds = 0.
    if row.data.adsorbate not in adsDistinctList: adsDistinctList.append(row.data.adsorbate)
    graphName = 'Mo2C_'+str(row.data.index)+'_'+str(row.data.iterm)+'_'+row.data.adsorbate+'_'+str(row.data.adsorbateIsomer)+'_'+str(row.data.adsorbateConfiguration)+'_'+str(row.id)
    nnNeighborSelfBond = True
    G = adsGraphGen(iniAtoms,adsIndices,graphName,row.energy) 
    Gf = adsGraphGen(finAtoms,adsIndices,graphName,row.energy)
    nx.write_graphml(G,iniDir+'/'+graphName+'.graphml')
    nx.write_graphml(Gf,finDir+'/'+graphName+'.graphml')
    graphNameList.append(graphName)
    y_bindingE.append(BE)
    y_totalE.append(row.energy)
    rowID.append(row.id)
    surfaceIndex.append(str(row.data.index))
    surfaceTerminations.append(str(row.data.iterm))
    adsList.append(row.data.adsorbate)
    adsIsomerList.append(str(row.data.adsorbateIsomer))
    adsConfigurationList.append(str(row.data.adsorbateConfiguration))
    adsIndicesList.append(adsIndices)
    distMaxList.append(distMax)
    distMaxAdsList.append(distMaxAds)
        
    
print("--- %s seconds ---" % (time.time() - start_time))